# Create Load data for disaggregration process of feeder level cooling and heating predictions

1. Import load data (from ResStock Parquet files in SMART-DS)
2. Filter parquet_files based on feeder's loads.dss 
3. Create columns: total demand, cooling sum, heating and non cooling-heating load
4. Save as dictionary, clustered by feeders (for disaggregration of feeder level cooling and heating predictions)

### Import packages

In [1]:
import numpy as np
import pyarrow.parquet as pq
import joblib
import scipy
import pandas as pd
from pandas import DatetimeIndex
import time
import os
import re
import glob
import gc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime, timedelta
import seaborn as sns
import pytz

import yaml
import pprint

from src import input_ops
from src import model_ops
from src import aux_ops

### Load config file with scenarios and parameters 

In [2]:
config_file_name = 'config1'; config_path = f"config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)
# pprint.pprint(config, sort_dicts=False) # print config

# lists of parameters to use for TGW df creation
TGW_scenario = config['TGW_scenario']
TGW_weather_years = config['TGW_weather_years']
prediction_model_str = config['prediction_model']
aggregation_level = config['aggregation_level']

X_columns = config[config['X_columns_set']]
CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']

input_data_dict_name = config['input_data_dict_name']
aggregation_level = config['aggregation_level']
smart_ds_year = config['smart_ds_years'][0]
building_types = config["building_types"]

input_data_training_path = config['input_data_training_path']
CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']
start_month = config['start_month']
end_month = config['end_month']

## Initialize parameters for saving paths
Y_column = config['Y_column']
input_data_prediction_path = config['input_data_prediction_path']
output_path_prediction_str = config['output_data_prediction_path']

smart_ds_year = config['smart_ds_years'][0]
smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}"

### Option 1: Create dictionary of measured building cooling and heating per feeder (from resstock parquet files)

In [3]:
start_time = time.time()

# Loop through all city, region, year, and building type combinations
for city, regions in CITY_REGIONS_TO_RUN.items():
    if aggregation_level != 'feeder':
        raise ValueError("This code supports feeder-level aggregation only - check aggregation_level value in config file")
    for region in regions:
        # Initialize dictionary to store region results
        measured_buildings_cool_heat_dict = {}
        ### Import and process smart-ds load data ###
        parquet_data_path = f'/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/load_data' # path to parquet files
        region_directory = f"/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss/" # path to where feeder folders are
        # Create a list of load_models (regional / feeders / buildings)   
        load_models = input_ops.make_feeder_list(region_directory) # list of feeders
        print(f'......Creating data frame for {smart_ds_year} {city} {region} ......')
        for load_model in load_models: # for feeder in feeders list
            for building_type in building_types:
                print(f'......Creating data frames for {smart_ds_year} {city} {region} feeder {load_model} {building_type}......')
                ### Get all parquet files from load_data folder (all res/com stock profiles in the region) ###
                parquet_files = sorted([f for f in os.listdir(parquet_data_path) if f.startswith(building_type) and f.endswith(".parquet")])
                feeder_path_name = input_ops.add_feeder_upper_folder(load_model)
                feeder_path = f'/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss/{feeder_path_name}'
                ## filter parquet_files list and keep only res/com stock profiles that exist in the feeder (Extract valid names from Loads.dss)  
                valid_names = set()
                file_path = feeder_path + "/Loads.dss"
                with open(file_path, "r") as f:
                    for line in f:
                        match = re.search(r"yearly=(res|com)_kw_(\d+)", line)
                        if match:
                            valid_names.add(f"{match.group(1)}_{match.group(2)}")  # e.g., Extract "res_376" or "com_10111"
                # Filter parquet_files based on feeder's loads.dss and skip to next feeder if non exist
                parquet_files = [f for f in parquet_files if f.replace(".parquet", "") in valid_names]
                if not parquet_files:
                    print(f"Feeder {feeder_path} skipped since no parquet files found in the given folder with the specified prefix.")                            
                    continue 
                    
                # Initialize dictionary for this feeder and building type
                measured_buildings_cool_heat_dict[(smart_ds_year, city, region, load_model, building_type)] = {}

                # Loop through all parquet files in feeder, process them and add to dictionary
                for file in parquet_files:
                    parquet_file_path = os.path.join(parquet_data_path, file)
                    load_prefix = file.split('_')[0]

                    if load_prefix not in {'com', 'res'}:
                        print(f"Parquet file {file} skipped since it didn't start with 'com' or 'res'")
                        continue

                    load_df = input_ops.get_parquet_load_data(parquet_data_path, parquet_file_path, start_month, end_month)
                    load_df = input_ops.convert_columns_to_CH_and_non_CH(load_df)
                    load_df = load_df.set_index('date_time')

                    building_id = file.split('.')[0]
                    measured_buildings_cool_heat_dict[(smart_ds_year, city, region, load_model, building_type)][building_id] = load_df

                    
#                 # Initialize buildings dictionary in feeder key
#                 measured_buildings_cool_heat_dict[(smart_ds_year, city, region, load_model, building_type)] = {}
#                 ### Add to buildings dict dataframe of first file in parquet_files, convert 15min resolution to 1h, convert time column name, add month column and filter so start_month - end_month ###

#                 # Add first building dataframe
#                 parquet_file_path = os.path.join(parquet_data_path, parquet_files[0])
#                 load_prefix = parquet_files[0].split('_')[0]
#                 if load_prefix not in {'com', 'res'}:
#                     print(f"Parquet file skipped since it didn't start with com or res")
#                     continue
#                 load_df = input_ops.get_parquet_load_data(parquet_data_path, parquet_file_path,start_month,end_month)
#                 load_df = input_ops.convert_columns_to_CH_and_non_CH(load_df)
#                 load_df = load_df.set_index('date_time')
#                 measured_buildings_cool_heat_dict[(smart_ds_year, city, region, load_model, building_type)][parquet_files[0].split('.')[0]] = load_df

#                 ## Loop through building parquet files in feeder, get data_frame and add to dictionary
#                 for file in parquet_files[1:]:
#                     parquet_file_path = os.path.join(parquet_data_path, file)
#                     load_prefix = file.split('_')[0]
#                     if load_prefix not in {'com', 'res'}:
#                         print(f"Parquet file skipped since it didn't start with com or res")
#                         continue
#                     load_df = input_ops.get_parquet_load_data(parquet_data_path, parquet_file_path,start_month,end_month)
#                     load_df = input_ops.convert_columns_to_CH_and_non_CH(load_df)
#                     load_df = load_df.set_index('date_time')
#                     measured_buildings_cool_heat_dict[(smart_ds_year, city, region, load_model, building_type)][file.split('.')[0]] = load_df
        # Save joblib for city-region
        print(f'saving joblib for city {city} region {region}')
        input_data_region_dir = f'{smart_ds_load_path}/{city}/{region}/buildings'
        os.makedirs(input_data_region_dir, exist_ok=True)     # Ensure the directory exists
        joblib.dump(measured_buildings_cool_heat_dict, os.path.join(input_data_region_dir, "measured_buildings_cool_heat_dict.joblib")) # Save the file
end_time = time.time(); print(f"Runtime for loading data: {(end_time - start_time) / 60:.2f} minutes")

......Creating data frame for 2018 SFO P2U ......
......Creating data frames for 2018 SFO P2U feeder p2uhs11_1247--p2udt2578 res......
......Creating data frames for 2018 SFO P2U feeder p2uhs11_1247--p2udt2578 com......
......Creating data frames for 2018 SFO P2U feeder p2uhs11_1247--p2udt1836 res......
......Creating data frames for 2018 SFO P2U feeder p2uhs11_1247--p2udt1836 com......
......Creating data frames for 2018 SFO P2U feeder p2uhs11_1247--p2udt65 res......
......Creating data frames for 2018 SFO P2U feeder p2uhs11_1247--p2udt65 com......
......Creating data frames for 2018 SFO P2U feeder p2uhs7_1247--p2udt5977 res......
......Creating data frames for 2018 SFO P2U feeder p2uhs7_1247--p2udt5977 com......
......Creating data frames for 2018 SFO P2U feeder p2uhs7_1247--p2udt9294 res......
......Creating data frames for 2018 SFO P2U feeder p2uhs7_1247--p2udt9294 com......
......Creating data frames for 2018 SFO P2U feeder p2uhs7_1247--p2udt5979 res......
......Creating data fram

In [4]:
city = 'SFO'
region = 'P2U'
input_data_region_dir = f'{smart_ds_load_path}/{city}/{region}/buildings'
measured_buildings_cool_heat_dict = joblib.load(os.path.join(input_data_region_dir, "measured_buildings_cool_heat_dict.joblib")) 
measured_buildings_cool_heat_dict.keys()

dict_keys([('2018', 'SFO', 'P2U', 'p2uhs11_1247--p2udt2578', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs11_1247--p2udt2578', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs11_1247--p2udt1836', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs11_1247--p2udt1836', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs11_1247--p2udt65', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs11_1247--p2udt65', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs7_1247--p2udt5977', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs7_1247--p2udt5977', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs7_1247--p2udt9294', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs7_1247--p2udt9294', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs7_1247--p2udt5979', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs7_1247--p2udt5979', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs9_1247--p2udt9130', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs9_1247--p2udt9130', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs9_1247--p2udt12222', 'res'), ('2018', 'SFO', 'P2U', 'p2uhs9_1247--p2udt12222', 'com'), ('2018', 'SFO', 'P2U', 'p2uhs9_1247--p2udt12221', 'res'), ('2018', 'SFO',

###  Option 2: Load measured buildings cooling heating dict (single region)

In [5]:
city = 'GSO'
region = 'rural'
smart_ds_year = config['smart_ds_years'][0]
smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}"
input_data_region_dir = f'{smart_ds_load_path}/{city}/{region}/buildings'
measured_buildings_cool_heat_dict = joblib.load(os.path.join(input_data_region_dir, "measured_buildings_cool_heat_dict.joblib")) # Save the file

measured_buildings_cool_heat_dict.keys()

dict_keys([('2018', 'GSO', 'rural', 'rhs2_1247--rdt1262', 'res'), ('2018', 'GSO', 'rural', 'rhs2_1247--rdt1262', 'com'), ('2018', 'GSO', 'rural', 'rhs2_1247--rdt1264', 'res'), ('2018', 'GSO', 'rural', 'rhs2_1247--rdt1264', 'com'), ('2018', 'GSO', 'rural', 'rhs0_1247--rdt1527', 'res'), ('2018', 'GSO', 'rural', 'rhs0_1247--rdt1527', 'com'), ('2018', 'GSO', 'rural', 'rhs0_1247--rdt1948', 'res'), ('2018', 'GSO', 'rural', 'rhs0_1247--rdt1948', 'com'), ('2018', 'GSO', 'rural', 'rhs0_1247--rdt1534', 'res'), ('2018', 'GSO', 'rural', 'rhs0_1247--rdt1534', 'com'), ('2018', 'GSO', 'rural', 'rhs1_1247--rdt137', 'res'), ('2018', 'GSO', 'rural', 'rhs1_1247--rdt137', 'com'), ('2018', 'GSO', 'rural', 'rhs3_1247--rdt2999', 'com'), ('2018', 'GSO', 'rural', 'rhs3_1247--rdt2705', 'res'), ('2018', 'GSO', 'rural', 'rhs3_1247--rdt2705', 'com')])

In [4]:
measured_buildings_cool_heat_dict[('2018', 'GSO', 'rural', 'rhs2_1247--rdt1262', 'res')].keys()

dict_keys(['res_101', 'res_1013', 'res_1036', 'res_1037', 'res_1041', 'res_1042', 'res_1070', 'res_1094', 'res_1108', 'res_1112', 'res_1113', 'res_1138', 'res_1148', 'res_1157', 'res_1160', 'res_1162', 'res_1186', 'res_1190', 'res_1193', 'res_1205', 'res_1209', 'res_1220', 'res_1223', 'res_1228', 'res_1258', 'res_1267', 'res_1270', 'res_1273', 'res_1276', 'res_128', 'res_1283', 'res_141', 'res_149', 'res_162', 'res_182', 'res_184', 'res_19', 'res_199', 'res_2', 'res_203', 'res_205', 'res_206', 'res_207', 'res_225', 'res_229', 'res_236', 'res_24', 'res_245', 'res_250', 'res_26', 'res_263', 'res_267', 'res_278', 'res_280', 'res_282', 'res_305', 'res_314', 'res_324', 'res_33', 'res_339', 'res_35', 'res_362', 'res_366', 'res_39', 'res_4', 'res_401', 'res_407', 'res_409', 'res_416', 'res_421', 'res_436', 'res_450', 'res_454', 'res_46', 'res_47', 'res_470', 'res_502', 'res_504', 'res_517', 'res_523', 'res_549', 'res_561', 'res_616', 'res_621', 'res_624', 'res_626', 'res_627', 'res_63', 'res_

In [7]:
measured_buildings_cool_heat_dict[('2018', 'GSO', 'rural', 'rhs2_1247--rdt1262', 'res')]['res_1013'].head(2)

,total_site_electricity_kw,cooling_sum_kw,heating_kw,non_cool_n_heat_kw
date_time,,,,
2018-01-01 00:00:00,1.546879,0.119552,0.482743,0.944584
2018-01-01 01:00:00,1.466327,0.116655,0.451855,0.897817
